In [6]:
import gtsam

In [2]:
#### FILTER OBJECT
class GraphTracker():
    def __init__(self):
        
        # Member variables
        self.init = False
        self.detections = []
        self.tracks = []
        self.graph = gtsam.NonlinearFactorGraph()
        self.trk_delete_thresh = 0.25
        self.trk_delete_age = 10
        self.trk_create_thresh = 0.75

    # Member functions
    def handle_detections(self,det_array_msg):
        self.detections = det_array_msg.boxes
        
        # TODO - Compute detection-track correspondences
        
        # TODO - Assign detections to track variables
        
        # TODO - create tracks as needed
        
        # Delete tracks as needed
        self.delete_tracks()
        
    def delete_tracks(self):
        self.tracks = [track for track in self.tracks if track.prob < self.trk_delete_thresh]
        self.tracks = [track for track in self.tracks if track.age > self.trk_delete_age]
        
    # birth
    # compute assignment
    # add factors
    # optimize

In [5]:
#### DETECTION OBJECT
class Detection():
    def __init__(self, det_msg):
        self.px = det_msg.pose.position.x
        self.py = det_msg.pose.position.y
        self.pz = det_msg.pose.position.z
        self.age = 0
        self.prob = det_msg.value
        self.class = det_msg.label
        # Can be empty/null/contain partial info
        # class conf % - float
        # ??? BB dimensions

#### TRACK OBJECT
class Track():
    def __init__(self, det):
        self.px = det.px
        self.py = det.py
        self.pz = det.pz
        self.vx = 0
        self.vy = 0
        self.vz = 0
        self.prob = 
        self.age = det.age
        # heading in world frame
        # self.class = categorical distribution!
        # ??? class conf %

In [ ]:
#### HELPER FUNCTIONS
# Similarity metric (detection, track)

# Form similarity matrix (detections, tracks) -> return similarity matrix
# - call similarity metric

# Hungarian/Munkres algo (similarity matrix) -> return assignments (T_match, D_match, T_unmatch (missed/occ), D_unmatch (false, new obj, extra detection)
